## Report historical seasonality for the past or upcoming month

Uplift results are defined as the factor of uplift, positive or negative, determined by the Stats Models seasonal_decompose function... if analysis results show a value of 1.0 and max seasonality determined by the function is +$5, then the uplift is $5. 

In [7]:
past = True             # If true looks at past 20 trading days, if false looks at next 20 trading days
insights = ['AMZN']     # Stocks to check insights 

Import necessary packages

In [11]:
import os
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm

Calculate 20 day seasoanlity

In [12]:
# Read files from folder
folder = '../daily_prices'
files = os.listdir(folder)

# Create dataframe
seasonal_uplifts = dict()

for i, file in enumerate(files):
    # Read file
    df = pd.read_csv(os.path.join(folder, file))

    if not df.isna().any().any() and len(df.index) >= 504:
        # Decompose
        result = sm.tsa.seasonal_decompose(df['Close'], model='additive', period=252, extrapolate_trend=25, two_sided=False)
        seasonal = result.seasonal

        # Impute outliers with mean
        z_scores = np.abs(stats.zscore(seasonal))
        outliers = z_scores > 3
        seasonal[outliers] = seasonal.mean()

        # normalize seasonality data
        mins, maxs = seasonal.min(), seasonal.max()
        seasonal = 2*(seasonal-mins)/(maxs-mins)-1

        # Get seasonality for next month (20 trading days)
        if past:
            seasonality = seasonal[-252:][-20:].reset_index(drop=True)
        else:
            seasonality = seasonal[-252:][:20].reset_index(drop=True)

        # Add to dataframe
        seasonal_uplifts[file[:-4]] = seasonality.to_list()

seasonal_uplifts = pd.DataFrame(seasonal_uplifts)

Report mean seasonality

In [13]:
# Calculate mean of each column
mean_values = seasonal_uplifts.mean()

# Create a new DataFrame with mean values and column names
avg_seasonal_uplifts = pd.DataFrame({
    'Stock': mean_values.index,
    '20_Day_Seasonality': mean_values.values
}).sort_values(by=['20_Day_Seasonality'], ascending=False).reset_index(drop=True)

for stock in insights:
    avg = round(avg_seasonal_uplifts[avg_seasonal_uplifts['Stock'] == stock]['20_Day_Seasonality'].values[0], 3)
    if past:
        print(f'{stock} previous 20 day seasonal uplift: {avg}')
    else:
        print(f'{stock} upcoming 20 day seasonal uplift: {avg}')

avg_seasonal_uplifts.head(10)

AMZN previous 20 day seasonal uplift: 0.161


,Stock,20_Day_Seasonality
0,MOH,0.787226
1,CBOE,0.768031
2,NRG,0.762124
3,MPC,0.741053
4,ROL,0.725566
5,CRM,0.688604
6,CTAS,0.657960
7,NFLX,0.656508
8,AJG,0.649945
9,ELV,0.638118
